In [2]:
with open('file1.txt', 'r') as f1:
    for row in f1:
        print(row, end='')
print('\n-------------------------------')
with open('file2.txt', 'r') as f2:
    for row in f2:
        print(row, end='')
print('\n-------------------------------')
with open('file3.txt', 'r') as f3:
    for row in f3:
        print(row, end='')
print('\n-------------------------------')



file1_line1
file1_line2
file1_line3
-------------------------------
file2_line1
file2_line2
file2_line3
-------------------------------
file3_line1
file3_line2
file3_line3
-------------------------------


In [4]:
with open('file1.txt', 'r') as f1, open('file2.txt', 'r') as f2, open('file3.txt', 'r') as f3:
    print(f1.readlines())
    print(f2.readlines())
    print(f3.readlines())



['file1_line1\n', 'file1_line2\n', 'file1_line3']
['file2_line1\n', 'file2_line2\n', 'file2_line3']
['file3_line1\n', 'file3_line2\n', 'file3_line3']


In [6]:
with open('file1.txt', 'r') as f1:
    with open('file2.txt', 'r') as f2:
        with open('file3.txt', 'r') as f3:
            print(f1.readlines())
            print(f2.readlines())
            print(f3.readlines())


['file1_line1\n', 'file1_line2\n', 'file1_line3']
['file2_line1\n', 'file2_line2\n', 'file2_line3']
['file3_line1\n', 'file3_line2\n', 'file3_line3']


In [4]:
from contextlib import contextmanager

In [5]:
@contextmanager
def open_file(fname):
    print(f'opening file {fname}...')
    f = open(fname)
    try:
        yield f
    finally:
        print(f'closing file {fname}...')
        f.close()

In [11]:
f_names = 'file1.txt', 'file2.txt', 'file3.txt'


exits = []
enters = []

for f_name in f_names:
    ctx = open_file(f_name)
    enters.append(ctx.__enter__)
    exits.append(ctx.__exit__)

In [12]:
files = [enter() for enter in enters]
print(files)

opening file file1.txt...
opening file file2.txt...
opening file file3.txt...
[<_io.TextIOWrapper name='file1.txt' mode='r' encoding='utf-8'>, <_io.TextIOWrapper name='file2.txt' mode='r' encoding='utf-8'>, <_io.TextIOWrapper name='file3.txt' mode='r' encoding='utf-8'>]


In [13]:
while True:
    try:
        rows = [next(f).strip('\n') for f in files]
    except StopIteration:
        break
    else:
        row = ','.join(rows)
        print(row)


file1_line1,file2_line1,file3_line1
file1_line2,file2_line2,file3_line2
file1_line3,file2_line3,file3_line3


In [15]:
for exit in exits[::-1]:
    exit(None, None, None)


closing file file3.txt...
closing file file2.txt...
closing file file1.txt...


In [17]:
f_names = 'file1.txt', 'file2.txt', 'file3.txt'


exits = []
enters = []

for f_name in f_names:
    ctx = open_file(f_name)
    enters.append(ctx.__enter__)
    exits.append(ctx.__exit__)

files = [enter() for enter in enters]

while True:
    try:
        rows = [next(f).strip('\n') for f in files]
    except StopIteration:
        break
    else:
        row = ','.join(rows)
        print(row)

for exit in exits[::-1]:
    exit(None, None, None)


opening file file1.txt...
opening file file2.txt...
opening file file3.txt...
file1_line1,file2_line1,file3_line1
file1_line2,file2_line2,file3_line2
file1_line3,file2_line3,file3_line3
closing file file3.txt...
closing file file2.txt...
closing file file1.txt...


In [6]:
class NestedContexts:
    def __init__(self, *contexts):
        self._enters = []
        self._exits = []
        self._values = []

        for ctx in contexts:
            self._enters.append(ctx.__enter__)
            self._exits.append(ctx.__exit__)

    def __enter__(self):
        for enter in self._enters:
            self._values.append(enter())
        return self._values

    def __exit__(self, exc_type, exc_value, traceback):
        for exit in self._exits[::-1]:
            exit(exc_type, exc_value, traceback)
        return False

In [7]:
with NestedContexts(open_file('file1.txt'), open_file('file2.txt'), open_file('file3.txt')) as files:
    for file in files:
        for row in file:
            print(row.strip('\n'))


opening file file1.txt...
opening file file2.txt...
opening file file3.txt...
file1_line1
file1_line2
file1_line3
file2_line1
file2_line2
file2_line3
file3_line1
file3_line2
file3_line3
closing file file3.txt...
closing file file2.txt...
closing file file1.txt...


In [8]:
f_names = 'file1.txt', 'file2.txt', 'file3.txt'

contexts = [open_file(f_name) for f_name in f_names]

with NestedContexts(*contexts) as files:
    for file in files:
        for row in file:
            print(row.strip('\n'))


opening file file1.txt...
opening file file2.txt...
opening file file3.txt...
file1_line1
file1_line2
file1_line3
file2_line1
file2_line2
file2_line3
file3_line1
file3_line2
file3_line3
closing file file3.txt...
closing file file2.txt...
closing file file1.txt...


In [9]:
class NestedContexts:
    def __init__(self):
        self._exits = []

    def __enter__(self):
        return self
    
    def enter_context(self, ctx):
        self._exits.append(ctx.__exit__)
        value = ctx.__enter__()
        return value

    def __exit__(self, exc_type, exc_value, traceback):
        for exit in self._exits[::-1]:
            exit(exc_type, exc_value, traceback)
        return False

In [13]:
with NestedContexts() as nested:
    f = [nested.enter_context(open_file(f_name)) for f_name in f_names]
    for file in f:
        for row in file:
            print(row.strip('\n'))

opening file file1.txt...
opening file file2.txt...
opening file file3.txt...
file1_line1
file1_line2
file1_line3
file2_line1
file2_line2
file2_line3
file3_line1
file3_line2
file3_line3
closing file file3.txt...
closing file file2.txt...
closing file file1.txt...


In [14]:
from contextlib import ExitStack

In [15]:
with ExitStack() as stack:
    files = [stack.enter_context(open_file(f_name)) for f_name in f_names]
    for file in files:
        for row in file:
            print(row.strip('\n'))


opening file file1.txt...
opening file file2.txt...
opening file file3.txt...
file1_line1
file1_line2
file1_line3
file2_line1
file2_line2
file2_line3
file3_line1
file3_line2
file3_line3
closing file file3.txt...
closing file file2.txt...
closing file file1.txt...
